In [1]:
import numpy as np
import pandas as pd
import requests
import yaml
import altair as alt

In [11]:
url = "https://api.ies.ed.gov/eric/"


def load_data(query):
    
    querystring1 = {"search": query, "format": "json", "fields": "publicationdateyear"}
    response1 = requests.request("GET", url, params=querystring1)
    text1 = yaml.safe_load(response1.text)
    results_max = text1["response"]["numFound"]
    print(results_max)
    
    df = []
    results = 0
    start = 0
    while results < results_max:
        querystring = {"search": query, "format": "json", "rows": 2000, "start": start, "fields": "publicationdateyear"}
        response = requests.request("GET", url, params=querystring)
        text = yaml.safe_load(response.text)
        df_mini = pd.DataFrame(text["response"]["docs"])
        df.append(df_mini)
        # keeping at least 30 results
        results += len(df_mini)
        start += 2000
    
    df = pd.concat(df)
    return df

In [12]:
df = load_data("psychometrics")

8880


In [13]:
df

,publicationdateyear
0,1970.0
1,1999.0
2,1971.0
3,1971.0
4,1971.0
...,...
875,2016.0
876,2016.0
877,2014.0
878,2011.0


In [ ]:
#df["publicationdateyear"].value_counts()
df = df["publicationdateyear"].value_counts().rename_axis('year').reset_index(name='counts')

In [ ]:
alt.Chart(df_summ).mark_area().encode(
    x="year:Q",
    y="counts:Q",
    tooltip=["counts"]
)